# Top K with Vector Stores

Tools:
1. LangChain: standardize way to implement (set up, create, and query) multiple vector stores
2. Vector Stores:
    1. Chroma
3. Embedding Models
    1. HuggingFace

[LangChain-Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma/)

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm
from uuid import uuid4

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.documents import Document

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from data_processing import DataProcessing

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Prediction and Observations Data

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)

log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)

col_name = 'Base Sentence'
predictions = DataProcessing.df_to_list(predictions_df, col_name)
observations = DataProcessing.df_to_list(observations_df, col_name)
len(predictions), len(observations)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/pr

(705, 1903)

## Embedding Model(s)

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

#

## Create Vector Store: Collect Name + Connect Embedding Model(s)

- Collection [store your embeddings, documents, and any additional metadata.][[Getting Started with Chroma](https://docs.trychroma.com/docs/overview/getting-started)]
- Collections index your embeddings and documents, and enable efficient retrieval and filtering.

In [5]:
vector_store = Chroma(
    collection_name="prediction_collection",
    embedding_function=embeddings,
)

## Create a Chroma Client

In [6]:
import chromadb

client = chromadb.Client()

## Add Observations to Vector Store

In [7]:
# observations_df.loc

In [8]:
documents = []

for index, row in tqdm(observations_df.iterrows()):
    # print(f"""Index: {index},
    #       Name: {row['Base Sentence']},
    #       Sentence Label: {row['Sentence Label']},
    #       Domain: {row['Domain']},
    #       Model Name: {row['Model Name']},
    #       API Name: {row['API Name']},
    #       Template Number: {row['Template Number']}
    #       """)
    idx = index
    base_sentence = row['Base Sentence']
    sentence_label = row['Sentence Label']
    domain = row['Domain']
    model_name = row['Model Name']
    api_name = row['API Name']
    template_number = row['Template Number']

    document = Document(
        page_content=base_sentence,
        metadata={"sentence_label": sentence_label,
                  "domain": domain,
                  "model_name": model_name,
                  "api_name": api_name,
                  "template_number": template_number
                  },
        id=idx,)
    
    documents.append(document)

# documents

1903it [00:00, 66159.07it/s]


In [9]:
uuids = [str(uuid4()) for _ in range(len(documents))]
# uuids

## Add Documents to Vector Store

In [10]:
vector_store.add_documents(documents=documents, ids=uuids)

['c5b9dcde-5118-4b7e-b757-6df4fc7759d9',
 '7b84f94b-7562-4ba5-a7ae-86104e6f6274',
 '55965c65-1be0-4c45-90ac-ea212fc30762',
 '8a8c87c2-1f44-41c7-950f-6f0b7d0fb771',
 '9b5c6f11-5d10-4602-a098-f67b5408d198',
 '81e9a6da-5961-464f-87db-7825a2447750',
 'f2196aa9-9bc6-4636-a3a4-df8b5242cd07',
 'dff26036-c08a-42d1-95b6-613a82b8ed38',
 '16106899-a034-402e-94ca-4d3a64dbde43',
 '212ff47a-3988-4b05-895e-bbf463fa7f65',
 '28617c1c-74a6-405d-8f94-aa65374bbc6a',
 'b22544b5-7f73-4780-a99d-548719fc130f',
 '11681884-fc10-48e6-beab-9772cff6b647',
 'c108295b-93da-4b51-884d-bad504f3b6fd',
 '053a0d17-a175-4567-8a71-e42c78b154de',
 '976f5c18-b8b8-44e5-84dc-5e2b3ca66b2f',
 '3d223fe7-c51e-4ee2-a987-14d8aec2de2e',
 '391d2bd8-fd02-4e89-8707-b8b973383b2b',
 'cc01807d-895f-4cbe-8b40-fbd92eef9b4a',
 'fdd9cdb4-7fba-415d-a7a2-a14a7ff074ae',
 '21cf6543-f053-4a20-9577-f128d6f68ff6',
 '5bf2b1f6-43da-4c72-ae0a-afadc491550e',
 '4fad0539-6e42-4c0f-b08c-03c78c94856f',
 'b007c023-cbe8-4f73-9cd9-e94c3312962f',
 '69104d5e-feec-

## Query Vector Store

In [11]:
predictions[:3]

['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
 'On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.',
 'Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.']

In [12]:
for idx, prediction in enumerate(predictions):
    print(idx, prediction)
    print("-------Similarity-------")
    results = vector_store.similarity_search(
        prediction,
        k=3,
    )
    for res in results:
        print(f"* {res.page_content} [{res.metadata}]")

    print()
    print("-------Similarity with score-------")
    results = vector_store.similarity_search_with_score(
        prediction, k=1,
    )
    for res, score in results:
        print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

    print()
    print("-------Similarity by vector-------")
    results = vector_store.similarity_search_by_vector(
        embedding=embeddings.embed_query(prediction), k=3
    )
    for doc in results:
        print(f"* {doc.page_content} [{doc.metadata}]")

    print()
    print("-------Retriever-------")
    retriever = vector_store.as_retriever(
        search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
    )
    retriever.invoke(prediction)

0 JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.
-------Similarity-------
* JPMorgan Chase noted on 21/08/2024, the revenue at Amazon fell. [{'api_name': 'GROQ_CLOUD', 'template_number': 3, 'model_name': 'llama-3.1-8b-instant', 'sentence_label': 0, 'domain': 'finance'}]
* On 09/15/2023, the senior analyst at JP Morgan noted that the gross profit at Amazon had decreased. [{'sentence_label': 0, 'template_number': 1, 'api_name': 'NAVI_GATOR', 'model_name': 'mistral-small-3.1', 'domain': 'finance'}]
* On 08/15/2024, the financial analyst at JPMorgan Chase observed that the net profit at Amazon had increased in Q2 2024. [{'model_name': 'llama-3.1-70b-instruct', 'api_name': 'NAVI_GATOR', 'template_number': 1, 'domain': 'finance', 'sentence_label': 0}]

-------Similarity with score-------
* [SIM=0.308853] JPMorgan Chase noted on 21/08/2024, the revenue at Amazon fell. [{'template_number': 3, 'sentence_label': 0, 'domain': 'finance', 'model_name': 'l

KeyboardInterrupt: 